<a href="https://colab.research.google.com/github/psygrammer/about_python_dl_from_scrach_2/blob/main/notebooks/ch03/ch03_word2vec_sol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3. word2vec (Solution)

* 싸이그래머 / 어바웃파이썬 : 딥러닝 NLP [1]
* 김무성

# 차례 
* 3.1 추론 기반 기법과 신경망
  - 3.1.1 통계 기반 기법의 문제점
  - 3.1.2 추론 기반 기법 개요
  - 3.1.3 신경망에서의 단어 처리
* 3.2 단순한 word2vec
  - 3.2.1 CBOW 모델의 추론 처리
  - 3.2.2 CBOW 모델의 학습
  - 3.2.3 word2vec의 가중치와 분산 표현
* 3.3 학습 데이터 준비
  - 3.3.1 맥락과 타깃
  - 3.3.2 원핫 표현으로 변환
* 3.4 CBOW 모델 구현
  - 3.4.1 학습 코드 구현
* 3.5 word2vec 보충
  - 3.5.1 CBOW 모델과 확률
  - 3.5.2 skip-gram 모델
  - 3.5.3 통계 기반 vs. 추론 기반
* 3.6 정리

------------------

In [1]:
!git clone https://github.com/psygrammer/about_python_dl_from_scrach_2.git

Cloning into 'about_python_dl_from_scrach_2'...
remote: Enumerating objects: 124, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 124 (delta 52), reused 33 (delta 5), pack-reused 0
Receiving objects: 100% (124/124), 753.41 KiB | 11.96 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [2]:
ls

about_python_dl_from_scrach_2/  sample_data/


In [3]:
cd /content/about_python_dl_from_scrach_2/notebooks/ch03

/content/about_python_dl_from_scrach_2/notebooks/ch03


In [4]:
ls

ch03_word2vec.ipynb  ch03_word2vec_sol.ipynb


------------------------

# 3.1 추론 기반 기법과 신경망
* 3.1.1 통계 기반 기법의 문제점
* 3.1.2 추론 기반 기법 개요
* 3.1.3 신경망에서의 단어 처리

## 3.1.1 통계 기반 기법의 문제점


## 3.1.2 추론 기반 기법 개요


## 3.1.3 신경망에서의 단어 처리

In [5]:
import numpy as np

In [6]:
# 입력
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
c

array([[1, 0, 0, 0, 0, 0, 0]])

In [7]:
# 가중치
# 랜덤 초기화
W = np.random.randn(7,3) 
W

array([[ 0.37904857, -0.17495786,  0.40460745],
       [-0.28343065, -0.47231923,  1.93093803],
       [-2.27192446,  0.16759212, -1.65126446],
       [ 0.36473416,  0.85314654,  0.40256707],
       [ 1.35969155, -0.2887706 ,  0.28834987],
       [-0.79640239,  1.39328258,  0.35621497],
       [ 0.56437476, -1.53958825,  1.78909353]])

In [8]:
# 중간노드
h = np.matmul(c, W) 
h

array([[ 0.37904857, -0.17495786,  0.40460745]])

#### 위의 코드는 1장에서 구현한 MaMul 계층으로도 수행할 수 있다. 

In [9]:
import sys
sys.path.append('..')

In [10]:
import numpy as np
from common.layers import MatMul

In [11]:
# 입력
c = np.array([[1, 0, 0, 0, 0, 0, 0]])
c

array([[1, 0, 0, 0, 0, 0, 0]])

In [12]:
# 가중치
# 랜덤 초기화
W = np.random.randn(7,3) 
W

array([[ 0.14166768,  0.27625725, -1.11224845],
       [-1.74901213, -0.27704002,  2.06917325],
       [-0.24361224,  0.09044699,  1.56806975],
       [-1.19274646, -0.10974046,  1.62684306],
       [-0.46053395,  1.10265044,  0.23546217],
       [-1.74780059, -1.22118024, -0.22836404],
       [ 0.19886821, -0.00606937, -0.63785956]])

In [13]:
layer = MatMul(W)
layer

In [14]:
h = layer.forward(c)
h

array([[ 0.14166768,  0.27625725, -1.11224845]])

-------------

# 3.2 단순한 word2vec
* 3.2.1 CBOW 모델의 추론 처리
* 3.2.2 CBOW 모델의 학습
* 3.2.3 word2vec의 가중치와 분산 표현


## 3.2.1 CBOW 모델의 추론 처리


In [15]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul

In [16]:
# 샘플 맥락 데이터
c0 = np.array([[1, 0, 0, 0, 0, 0, 0]])
c1 = np.array([[0, 0, 1, 0, 0, 0, 0]])

In [17]:
# 가중치 초기화
W_in = np.random.randn(7, 3)
W_out = np.random.randn(3, 7)

In [18]:
# 계층 생성
in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

In [19]:
# 순전파
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 * (h0 + h1)
s = out_layer.forward(h)
print(s)

[[ 1.27882998  0.95715812 -0.69035553 -1.55040774  0.72225556  0.61256832
  -1.33145412]]


## 3.2.2 CBOW 모델의 학습


## 3.2.3 word2vec의 가중치와 분산 표현

--------------

# 3.3 학습 데이터 준비
* 3.3.1 맥락과 타깃
* 3.3.2 원핫 표현으로 변환


## 3.3.1 맥락과 타깃


In [20]:
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정

In [21]:
from common.util import preprocess

In [22]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [23]:
corpus

array([0, 1, 2, 3, 4, 1, 5, 6])

In [24]:
word_to_id

{'.': 6, 'and': 3, 'goodbye': 2, 'hello': 5, 'i': 4, 'say': 1, 'you': 0}

In [25]:
id_to_word

{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}

In [26]:
def create_contexts_target(corpus, window_size=1):
    '''맥락과 타깃 생성
    :param corpus: 말뭉치(단어 ID 목록)
    :param window_size: 윈도우 크기(윈도우 크기가 1이면 타깃 단어 좌우 한 단어씩이 맥락에 포함)
    :return:
    '''
    target = corpus[window_size:-window_size]
    contexts = []

    for idx in range(window_size, len(corpus)-window_size):
        cs = []
        for t in range(-window_size, window_size + 1):
            if t == 0:
                continue
            cs.append(corpus[idx + t])
        contexts.append(cs)

    return np.array(contexts), np.array(target)

In [27]:
context, target = create_contexts_target(corpus, window_size=1)

In [28]:
context

array([[0, 2],
       [1, 3],
       [2, 4],
       [3, 1],
       [4, 5],
       [1, 6]])

In [29]:
target

array([1, 2, 3, 4, 1, 5])

## 3.3.2 원핫 표현으로 변환

In [30]:
def convert_one_hot(corpus, vocab_size):
    '''원핫 표현으로 변환
    :param corpus: 단어 ID 목록(1차원 또는 2차원 넘파이 배열)
    :param vocab_size: 어휘 수
    :return: 원핫 표현(2차원 또는 3차원 넘파이 배열)
    '''
    N = corpus.shape[0]

    if corpus.ndim == 1:
        one_hot = np.zeros((N, vocab_size), dtype=np.int32)
        for idx, word_id in enumerate(corpus):
            one_hot[idx, word_id] = 1

    elif corpus.ndim == 2:
        C = corpus.shape[1]
        one_hot = np.zeros((N, C, vocab_size), dtype=np.int32)
        for idx_0, word_ids in enumerate(corpus):
            for idx_1, word_id in enumerate(word_ids):
                one_hot[idx_0, idx_1, word_id] = 1

    return one_hot

In [31]:
vocab_size = len(word_to_id)
vocab_size

7

In [32]:
target = convert_one_hot(target, vocab_size)
target

array([[0, 1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0]], dtype=int32)

In [33]:
context = convert_one_hot(context, vocab_size)
context

array([[[1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0]],

       [[0, 0, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0]],

       [[0, 0, 0, 1, 0, 0, 0],
        [0, 1, 0, 0, 0, 0, 0]],

       [[0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 0]],

       [[0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1]]], dtype=int32)

--------------

# 3.4 CBOW 모델 구현
* 3.4.1 학습 코드 구현


In [34]:
import sys
sys.path.append('..')
import numpy as np
from common.layers import MatMul, SoftmaxWithLoss


In [35]:
class SimpleCBOW:
    def __init__(self, vocab_size, hidden_size):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(H, V).astype('f')

        # 계층 생성
        self.in_layer0 = MatMul(W_in)
        self.in_layer1 = MatMul(W_in)
        self.out_layer = MatMul(W_out)
        self.loss_layer = SoftmaxWithLoss()

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer0, self.in_layer1, self.out_layer]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h0 = self.in_layer0.forward(contexts[:, 0])
        h1 = self.in_layer1.forward(contexts[:, 1])
        h = (h0 + h1) * 0.5
        score = self.out_layer.forward(h)
        loss = self.loss_layer.forward(score, target)
        return loss

    def backward(self, dout=1):
        ds = self.loss_layer.backward(dout)
        da = self.out_layer.backward(ds)
        da *= 0.5
        self.in_layer1.backward(da)
        self.in_layer0.backward(da)
        return None

## 3.4.1 학습 코드 구현

-----------------

# 3.5 word2vec 보충
* 3.5.1 CBOW 모델과 확률
* 3.5.2 skip-gram 모델
* 3.5.3 통계 기반 vs. 추론 기반


## 3.5.1 CBOW 모델과 확률


## 3.5.2 skip-gram 모델


## 3.5.3 통계 기반 vs. 추론 기반

-----------------

# 3.6 정리

------------------
# 참고문헌
* [1] 밑바닥부터 시작하는 딥러닝 2
  - https://www.hanbit.co.kr/store/books/look.php?p_code=B8950212853
  - github - https://github.com/WegraLee/deep-learning-from-scratch-2